<a href="https://colab.research.google.com/github/clrkwng/MusicGenerationProject/blob/master/Music_Generation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@test {"output": "ignore"}

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)


Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
     |████████████████████████████████| 1.5MB 9.2MB/s 
     |████████████████████████████████| 204kB 54.7MB/s 
     |████████████████████████████████| 2.3MB 42.0MB/s 
     |████████████████████████████████| 1.6MB 22.9MB/s 
     |████████████████████████████████| 2.3MB 55.4MB/s 
     |████████████████████████████████| 983kB 61.6MB/s 
     |█████████████████████████████

In [0]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# with open('/content/drive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat /content/drive/My\ Drive/foo.txt

In [0]:
midiSource = "/content/gdrive/My Drive/MusicGenFolder/mozart_eine_kleine.mid"
print(midiSource)


/content/gdrive/My Drive/MusicGenFolder/mozart_eine_kleine.mid


In [0]:
noteSeq = mm.midi_file_to_note_sequence(midiSource);

In [0]:
mm.play_sequence(noteSeq)
mm.plot_sequence(noteSeq)

In [0]:
print('Downloading model bundle. This will take less than a minute...')
mm.notebook_utils.download_bundle('basic_rnn.mag', '/content/')
bundle = mm.sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')

# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2


In [0]:
# Initialize the model.
print("Initializing Melody RNN...")
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print ('🎉 Done!')


Initializing Melody RNN...
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmp68bwdmpl/model.ckpt
🎉 Done!


In [0]:
num_steps = 500
temperature = 1.0

last_end_time = (max(n.end_time for n in noteSeq.notes) 
                  if noteSeq.notes else 0)
qpm = noteSeq.tempos[0].qpm
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
    start_time=last_end_time + seconds_per_step,
    end_time=total_seconds)


sequence = melody_rnn.generate(noteSeq, generator_options)
mm.plot_sequence(sequence)
mm.play_sequence(sequence, synth=mm.fluidsynth)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -489.116669 
